In [2]:
'''
__author__ : Hanwen Zha
__description__: Function for attach a node to the general taxonomy
__latest_update__: 2/27/2018
'''

'\n__author__ : Hanwen Zha\n__description__: Function for attach a node to the general taxonomy\n__latest_update__: 2/27/2018\n'

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from gensim.models import word2vec
from collections import Counter
from tqdm import tqdm_notebook
import re
from pprint import pprint

In [5]:
from util import ES
from util import rmTag_concat
from util import wrapTag
from util import rmUnderscore
from util import cleanText

In [6]:
root_dir = '/home/hanwen/Desktop/demo/NSCTA-Demo'

In [7]:
# load word embedding
file_word2vec = '{}/concept_based_retrieval/output/SEMANTIC_SCHOLAR120w/wordvec'.format(root_dir)
model = word2vec.Word2Vec.load(file_word2vec)

In [75]:
file_word2vec = '{}/embedding/word2vec_tuned_pretrained.bin'.format(root_dir)
model2 = word2vec.Word2Vec.load(file_word2vec)

In [8]:
# load taxonomy
with open('{}/webUI/static/tree.json_bak'.format(root_dir)) as f_in:
    taxonomy_data = json.load(f_in)

In [9]:
import sys
import spacy
import json
from tqdm import tqdm
# import ipdb
from spacy.tokens import Doc
from spacy.matcher import PhraseMatcher

try:
  unicode;
except NameError as e:
  unicode = str

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(' ')
        # All tokens 'own' a subsequent space character in this tokenizer
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)

nlp = spacy.load('en', disable=['ner', 'parser', 'tagger'])
nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

In [417]:
class taxonomy_tree():
    def __init__(self):
        self.tree_list = []
    def parse_tree(self, node, level):
        if level == 0:
            self.tree_list.append({'name':node['name'], 'keywords':node['keywords'], 'level':level})
        if 'children' not in node:
            self.tree_list.append({'name':node['name'], 'keywords':node['keywords'], 'level':level})
            return
        if 'children' in node:
            self.tree_list.append({'name':node['name'], 'keywords':node['keywords'], 'level':level})
            for child in node['children']:
                n = self.parse_tree(child, level+1)
    def output_tree_by_level(self):
        output_tree_list = []
        for lvl in range(4):
            for node in self.tree_list:
                if node['level'] == lvl:
                    output_tree_list.append(node)
        return output_tree_list

In [418]:
def flatten(matched_phrase_docs):
    flat_matched_phrase_docs = []
    for matched_phrase_doc in matched_phrase_docs:
        for doc in matched_phrase_doc:
            flat_matched_phrase_docs.append(doc)
    return flat_matched_phrase_docs

def flatten_once(matched_phrase_docs):
    flat_matched_phrase_docs = []
    for matched_phrase_doc in matched_phrase_docs:
        for doc in set(matched_phrase_doc):
            flat_matched_phrase_docs.append(doc)
    return flat_matched_phrase_docs

In [419]:
tree = taxonomy_tree()
tree.parse_tree(taxonomy_data, 0)
tree_list = tree.output_tree_by_level()
# tree_list = tree.tree_list

In [421]:
general_taxonomy_phrases = []
general_taxonomy_keywords = []
for node in tree_list:
    general_taxonomy_phrases.append(node['name'])
    general_taxonomy_keywords.extend(node['keywords'])

In [422]:
general_taxonomy_phrases.remove('quantum')

In [423]:
general_taxonomy_phrases = [rmUnderscore(node) for node in general_taxonomy_phrases]
general_taxonomy_keywords = [rmUnderscore(node) for node in general_taxonomy_keywords] 

In [424]:
INDEX_NAME = "taxongen1"
TYPE_NAME = "taxongen_docs"
es = ES(index_name=INDEX_NAME, type_name=TYPE_NAME)

In [425]:
# es2 = ES(index_name=INDEX_NAME, type_name=TYPE_NAME, size=1)
# total_dict = Counter()
# total_dict_loose = Counter()

# for phrase in tqdm_notebook(set(general_taxonomy_phrases + general_taxonomy_keywords)):
# #     _, total = es2.search_articles_by_phrase(phrase.replace('_', ' '))
#     _, total_loose = es2.search_articles_by_common_term(phrase.replace('_', ' '))
#     total_dict[phrase.replace(' ', '_')] = total
#     total_dict_loose[phrase.replace(' ', '_')] = total_loose

# import json

# with open('total_dict.json', 'w') as f:
#     json.dump(total_dict, f)

# with open('total_dict_loose.json', 'w') as f:
#     json.dump(total_dict_loose, f)

A Jupyter Widget

Phrase: low dimensional, Total hits = 167
Phrase: low dimensional, Total hits = 6498
Phrase: key agreement protocol, Total hits = 209
Phrase: key agreement protocol, Total hits = 652
Phrase: xml, Total hits = 4372
Phrase: xml, Total hits = 10930
Phrase: bioinformatics, Total hits = 1109
Phrase: bioinformatics, Total hits = 5549
Phrase: face detection and recognition, Total hits = 20
Phrase: face detection and recognition, Total hits = 1409
Phrase: web pages, Total hits = 917
Phrase: web pages, Total hits = 10410
Phrase: google, Total hits = 445
Phrase: google, Total hits = 3963
Phrase: ddh assumption, Total hits = 4
Phrase: ddh assumption, Total hits = 803
Phrase: keys, Total hits = 5619
Phrase: keys, Total hits = 79329
Phrase: feed forward neural networks, Total hits = 95
Phrase: feed forward neural networks, Total hits = 1632
Phrase: image processing applications, Total hits = 60
Phrase: image processing applications, Total hits = 11889
Phrase: kohonen, Total hits = 173
Phrase: kohon

Phrase: structured documents, Total hits = 10277
Phrase: development managment, Total hits = 14
Phrase: development managment, Total hits = 40715
Phrase: multilayer perceptrons, Total hits = 261
Phrase: multilayer perceptrons, Total hits = 2313
Phrase: thai, Total hits = 198
Phrase: thai, Total hits = 409
Phrase: dec pomdps, Total hits = 27
Phrase: dec pomdps, Total hits = 1264
Phrase: bisimulation, Total hits = 424
Phrase: bisimulation, Total hits = 1348
Phrase: spatial data, Total hits = 535
Phrase: spatial data, Total hits = 17403
Phrase: marketplace, Total hits = 415
Phrase: marketplace, Total hits = 2271
Phrase: spiking, Total hits = 1576
Phrase: spiking, Total hits = 4071
Phrase: artificial intelligence, Total hits = 940
Phrase: artificial intelligence, Total hits = 8326
Phrase: manifold, Total hits = 1478
Phrase: manifold, Total hits = 5544
Phrase: belief change, Total hits = 86
Phrase: belief change, Total hits = 1373
Phrase: minds, Total hits = 615
Phrase: minds, Total hits = 

limit_output extension: Maximum message size of 10000 exceeded with 12203 characters

In [390]:
with open('total_dict.json') as fin:
    total_dict = json.load(fin)
with open('total_dict_loose.json') as fin:
    total_dict_loose = json.load(fin)

In [392]:
query_generator = (i for i in general_taxonomy_phrases)

In [401]:
isprint = False
# query = next(query_generator)
query = 'quantum_learning'

ids, total = es.search_articles_by_phrase(query.replace('_', ' '))

ids_loose, total_loose = es.search_articles_by_common_term(query.replace('_', ' '))

texts = es.search_articles_by_ids(ids)
texts = [cleanText(line) for line in texts]

texts_loose = es.search_articles_by_ids(ids_loose)
texts_loose = [cleanText(line) for line in texts_loose]

sim_list = []
for node in tree_list:
    if node['level'] >= 4:
        continue
    try:
        sim = model.wv.similarity(wrapTag(node['name']), wrapTag(query))
        sim_list.append([node['name'], sim])
    except KeyError as e:
        sim_list.append([node['name'], 0])

if isprint:
    print('\n')
    print('sim_list -->')
    pprint(sim_list[:10])

sorted_sim_list = sorted(sim_list, key=lambda x:-x[1])
if isprint:
    print('\n')
    print('sorted_sim_list -->')
    pprint(sorted_sim_list[:10])

# these phrases are limited
phrases = general_taxonomy_phrases

# build phrase matcher
matcher = PhraseMatcher(nlp.vocab)
for v in phrases:
    matcher.add(v, None, nlp(unicode(v))) # python3 is unicode

# matcher.add(v, None, nlp(unicode(query.replace('_', ' '))))

phrase_docs = []
for text in tqdm_notebook(texts):
    doc = nlp(text.lower())
    matches = matcher(doc)
    nps = []
    prev_st = 0
    prev_ed = 0
    for m in matches:
        tokens = text.lower().split(' ')
        st = m[1]
        ed = m[2]
        if ed > prev_ed:
            np = {'st':st, 'ed':ed, 'text': '_'.join(tokens[st:ed])}
            nps.append(np)
            prev_ed = ed
    phrase_docs.append([np['text'] for np in nps])

phrase_docs_loose = []
for text in tqdm_notebook(texts_loose):
    doc = nlp(text.lower())
    matches = matcher(doc)
    nps = []
    prev_st = 0
    prev_ed = 0
    for m in matches:
        tokens = text.lower().split(' ')
        st = m[1]
        ed = m[2]
        if ed > prev_ed:
            np = {'st':st, 'ed':ed, 'text': '_'.join(tokens[st:ed])}
            nps.append(np)
            prev_ed = ed
    phrase_docs_loose.append([np['text'] for np in nps])

flat_matched_phrase_docs = flatten(phrase_docs)
flat_matched_phrase_docs_loose = flatten(phrase_docs_loose)

coocur_cnter = Counter(flat_matched_phrase_docs)
coocur_cnter_loose = Counter(flat_matched_phrase_docs_loose)

# -------------------
flat_matched_phrase_docs = flatten_once(phrase_docs)
flat_matched_phrase_docs_loose = flatten_once(phrase_docs_loose)
coocur_cnter_once = Counter(flat_matched_phrase_docs)
coocur_cnter_loose_once = Counter(flat_matched_phrase_docs_loose)

if isprint:
    print('\n')
    print('coocur_cnter -->')
    pprint(coocur_cnter.most_common(20))

    print('\n')
    print('coocur_cnter_loose -->')
    
    pprint(coocur_cnter_loose.most_common(20))


    
    print('\n')
    print('coocur_cnter -->')
    pprint(coocur_cnter_once.most_common(20))

    print('\n')
    print('coocur_cnter_loose -->')
    
    pprint(coocur_cnter_loose_once.most_common(20))

Phrase: quantum learning, Total hits = 1
Phrase: quantum learning, Total hits = 274


A Jupyter Widget

A Jupyter Widget

In [402]:
if total < 500:
    use_dict = total_dict_loose
    use_total = total_loose
    use_coour_counter_once = coocur_cnter_loose_once
else:
    use_dict = total_dict
    use_total = total
    use_coour_counter_once = coocur_cnter_once
    
norm = sum([p[1] for p in use_coour_counter_once.most_common(10)])
    
for phrase, freq in use_coour_counter_once.most_common(10):
#     flag = 0
#     freq/min(use_dict.get(phrase, 100000), use_total, 10000) >= ratio_threshold and
    if freq < hard_threshold:  
        continue
    try:
        embeding_sim = model.similarity(wrapTag(phrase), wrapTag(query))
    except:
        embeding_sim = None
    if embeding_sim and embeding_sim < 0.3:
        continue
        
    print(phrase, freq, use_total, use_dict[phrase], end=' ')
    print(round(embeding_sim, 2), end = ' ')
    print(round(freq/norm, 2))
#     flag = 'P'
#     if embeding_sim:
#         if embeding_sim >= 0.6:
#             flag = 'S'
            

machine_learning 40 274 21728 0.45 0.29
neural_network 34 274 30504 0.39 0.25
cryptography 23 274 5003 0.41 0.17
oblivious_transfer 9 274 665 0.49 0.07
reasoning 9 274 64993 0.45 0.07
information_retrieval 5 274 23896 0.33 0.04
geometry 5 274 17466 0.39 0.04


In [380]:
ratio_threshold = 0.05
hard_threshold = 5

In [77]:
# # coocur by search
# with open('../../data/query_docs/{}.txt'.format(query)) as fin:
#     texts_loose = []
#     for line in fin:
#         line = line.replace('_', ' ')
#         line = cleanText(line)
#         if line:
#             texts_loose.append(line)

In [55]:
# texts_keep_phrases = []
# for line in texts:
#     words = line.strip().split()
#     words = [w for w in words if '_' in w]
#     texts_keep_phrases.append(words)

In [103]:
# get phrase list
# only fit for quantum learning and quantum cryptography
# text file need to be prepared first

# phrases = []
# with open('{}/local-embedding/data/{}/input/keywords.txt'.format(root_dir, query)) as fin:
#     for line in fin:
#         phrase = line.lower().strip().split('_')
#         phrase = ' '.join(phrase)
#         phrases.append(phrase)

# phrases = list(set(phrases + general_taxonomy_phrases))
# phrases = [unicode(p) for p in phrases]

# get query relevant corpus
# only fit for quantum learning and quantum cryptography
# text file need to be prepared first

# texts = []
# with open('{}/local-embedding/data/{}/input/papers.txt'.format(root_dir, query)) as fin:
#     for line in fin:
#         line = line.replace('_', ' ')
#         line = cleanText(line)
#         if line:
#             texts.append(line)

# with open('{}/data/query_docs/arxiv/{}_with_fulltext.json'.format(root_dir, '_AND_'.join(query.split('_')))) as fin:
#     for line in fin:
#         line = json.loads(line)
#         abstract = line['abstract']
#         abstract = cleanText(abstract)
#         if abstract:
#             texts.append(abstract)

# # coocur by search (common term)
# with open('../../data/query_docs/{}.txt'.format(query)) as fin:
#     texts_loose = []
#     for line in fin:
#         line = cleanText(line)
#         if line:
#             texts_loose.append(line)

In [108]:
matched_phrase_docs = []
for phrase_d in phrase_docs:
    if query in phrase_d:
        matched_phrase_docs.append([d.lower() for d in phrase_d])

In [109]:
# print(matched_phrase_docs)

In [111]:
flat_matched_phrase_docs = flatten(matched_phrase_docs)
flat_matched_phrase_docs_loose = flatten(phrase_docs_loose)

In [112]:
coocur_cnter = Counter(flat_matched_phrase_docs)

coocur_cnter_loose = Counter(flat_matched_phrase_docs_loose)
coocur_cnter_loose.most_common(20)

In [114]:
for phrase, score in sorted_sim_list:
    if coocur_cnter[phrase]:
        print(phrase, score, coocur_cnter[phrase])

bdi 1.0 674
intelligent_agents 0.615422694555 23
reasoning 0.527531676139 113
virtual_agents 0.521432768301 3
cognition 0.506350408816 1
logic_programming 0.479229724513 2
reinforcement_learning 0.441630086817 3
pomdps 0.440560558399 5
ontologies 0.434279850902 1
markov_decision_processes 0.41674360131 2
planner 0.40754569657 6
semantic_web 0.407159572861 10
natural_language 0.359362183561 1
requirements_engineering 0.338489398075 3
emergency_response 0.3382106412 2
intrusion_detection 0.272758775944 2
graphical_models 0.251794794745 1
information_retrieval 0.24297572823 2
neural_network 0.185705947814 5
web_content 0.165298131255 3
manufacturing_system 0 3


In [115]:
for phrase, score in sorted_sim_list:
    if coocur_cnter_loose[phrase]:
        print(phrase, score, coocur_cnter_loose[phrase])

bdi 1.0 1203
intelligent_agents 0.615422694555 69
multi_agent_systems 0.553742866717 4
reasoning 0.527531676139 391
virtual_agents 0.521432768301 14
cognition 0.506350408816 13
logic_programming 0.479229724513 20
reinforcement_learning 0.441630086817 10
pomdps 0.440560558399 8
ontologies 0.434279850902 10
intelligent_decision_support_systems 0.432681444238 1
semantic_web_services 0.430769278309 1
markov_decision_processes 0.41674360131 3
planner 0.40754569657 8
semantic_web 0.407159572861 26
social_interaction 0.404729673139 1
swarm_intelligence 0.369946202315 1
natural_language 0.359362183561 5
pervasive_computing 0.346570987073 1
path_planning 0.340350956931 1
access_control 0.340031544903 3
requirements_engineering 0.338489398075 3
emergency_response 0.3382106412 2
business_intelligence 0.311680722299 1
computer_science 0.310400427224 9
machine_learning 0.281612373504 2
intrusion_detection 0.272758775944 2
graphical_models 0.251794794745 1
intelligent_transportation 0.244349306154 2

In [123]:
# model.wv.most_similar(wrapTag('deep learning'))

In [127]:
# model.wv.similar_by_vector(model.wv['quantum']+model.wv['wells'], topn=40)